In [1]:
import os

In [5]:
from langchain_community.chat_models.ollama import ChatOllama
llm = ChatOllama(model="llama3")

In [50]:
print(llm.invoke("Tell me a joke"))

content="Here's one:\n\nWhy don't eggs tell jokes?\n\n(Wait for it...)\n\nBecause they'd crack each other up!\n\nHope that made you smile! Do you want to hear another one?" response_metadata={'model': 'llama3', 'created_at': '2024-06-05T06:02:20.115529Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1822168541, 'load_duration': 1097389958, 'prompt_eval_count': 13, 'prompt_eval_duration': 87941000, 'eval_count': 39, 'eval_duration': 635511000} id='run-cee0b5cb-7486-48ec-9a90-769e581617ef-0'


In [7]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


## Basic RAG

In [10]:

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
embeddings = OllamaEmbeddings(model="llama3")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()



In [11]:

# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [14]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})

In [17]:
response

{'input': 'What is Task Decomposition?',
 'context': [Document(page_content='The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": dependency_task_ids, "args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes the id of the previous task which generates a new resource that the current task relies on. A special tag "-task_id" refers to the generated text image, audio and video in the dependency task with id as task_id. The task MUST be selected from the following options: {{ Available Task List }}. There is a logical relationship between tasks, please note their order. If the user input can\'t be parsed, you need to reply empty JSON. Here are several cases for your reference: {{ Demonstrations }}. The chat history is recorded as {{ Chat History }}. From this chat history, you can find the path of the user-mentioned resources for your task planning.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-

## Conversation RAG

In [18]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [20]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [21]:
## Manually updating Chat history

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

There are several common ways to perform task decomposition:

1. **Divide-and-Conquer**: Break down the task into smaller subtasks that can be solved independently. This approach is effective for tasks with multiple components or steps.
2. **Goal-Task-Step (GTS)**: Identify the goal, then break it down into specific tasks, and finally decompose each task into individual steps.
3. **Objectives-Operations-Outcomes (OOO)**: Define the objectives, identify the operations required to achieve them, and determine the outcomes or results expected.
4. **Decomposition by Function**: Identify the functional components of the task and break it down accordingly. For example, if the task involves data processing, you might decompose it into data cleaning, transformation, and analysis steps.
5. **Hierarchical Decomposition**: Break down the task into smaller subtasks that are further decomposed until you reach the individual steps required to complete the task.

These approaches can be applied in var

In [23]:
## Auto update chat history

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [24]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is the process of breaking down complex tasks into smaller, more manageable subtasks that can be solved individually or in parallel. This approach helps to simplify the task, reduce its complexity, and make it easier to solve.\n\nIn the context of AI-powered assistants like myself, task decomposition allows us to:\n\n1. Identify the dependencies between subtasks\n2. Focus on specific skills or knowledge required for each subtask\n3. Reuse solutions or partial solutions from previous tasks\n\nBy decomposing a complex task into smaller parts, we can leverage our abilities to solve each subtask and then combine the results to complete the original task.\n\nTask decomposition is particularly useful when dealing with tasks that require:\n\n1. Domain-specific knowledge or expertise\n2. Complex decision-making or problem-solving\n3. Integration of multiple skills or tools\n\nIn your case, the experiment on fine-tuning LLMs for arithmetic operations is an excellent example 

In [25]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"There are several approaches to task decomposition, and I'll outline some common ones:\n\n1. **Divide and Conquer**: Break down the complex task into smaller, more manageable subtasks that can be solved individually. This approach is effective when dealing with tasks that have a clear hierarchical structure or can be partitioned based on specific features.\n2. **Top-Down Approach**: Start by identifying the overall goal or objective of the task and then work your way down to identify the necessary steps or subtasks required to achieve it. This approach helps in capturing the broader context and ensuring that all relevant aspects are considered.\n3. **Bottom-Up Approach**: Begin with individual, smaller tasks or subtasks that can be solved independently, and then combine the results to form the overall solution. This approach is useful when dealing with tasks that have many local optima or where small changes can have significant effects.\n4. **Hybrid Approach**: Combine elements of bo

In [30]:
store['abc123'].messages

[HumanMessage(content='What is Task Decomposition?'),
 AIMessage(content='Task decomposition is the process of breaking down complex tasks into smaller, more manageable subtasks that can be solved individually or in parallel. This approach helps to simplify the task, reduce its complexity, and make it easier to solve.\n\nIn the context of AI-powered assistants like myself, task decomposition allows us to:\n\n1. Identify the dependencies between subtasks\n2. Focus on specific skills or knowledge required for each subtask\n3. Reuse solutions or partial solutions from previous tasks\n\nBy decomposing a complex task into smaller parts, we can leverage our abilities to solve each subtask and then combine the results to complete the original task.\n\nTask decomposition is particularly useful when dealing with tasks that require:\n\n1. Domain-specific knowledge or expertise\n2. Complex decision-making or problem-solving\n3. Integration of multiple skills or tools\n\nIn your case, the experime